<a href="https://colab.research.google.com/github/sjkim-audio/audio-dsp-lab/blob/main/notebooks/guitar_bass_separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import files
from IPython.display import Audio, display

def run_separator():
    # 1. 환경 설정 및 라이브러리 설치함
    # 최신 torchaudio와 호환성 문제 해결을 위해 버전 고정 (<2.6.0)
    # soundfile: 오디오 저장을 위한 백엔드 라이브러리
    print("⏳ 환경 설정 및 라이브러리 설치 중... (약 30초~1분)")
    get_ipython().system('pip install -q "torchaudio<2.6.0" soundfile')
    get_ipython().system('pip install -q demucs')
    print("✅ 설치 완료")

    # 2. 파일 업로드함
    print("\n📂 분리할 오디오 파일을 선택하세요 (.mp3, .wav 등)")
    uploaded = files.upload()

    if not uploaded:
        print("❌ 업로드된 파일이 없음. 코드 다시 실행 필요")
        return

    filename = list(uploaded.keys())[0]

    # 3. Demucs 음원 분리 실행함
    # -n htdemucs_6s: 6개 트랙(Bass, Guitar, Drums, Piano, Vocals, Other) 분리 모델
    print(f"\n🚀 '{filename}' 분석 및 분리 시작함 (GPU 권장)...")
    get_ipython().system(f'demucs -n htdemucs_6s "{filename}"')

    # 4. 결과 경로 찾기
    # Demucs 출력 기본 경로: separated/htdemucs_6s/파일명(확장자제외)/
    name_no_ext = os.path.splitext(filename)[0]
    output_dir = os.path.join('separated', 'htdemucs_6s', name_no_ext)

    # 5. 결과 재생 및 출력함
    if os.path.exists(output_dir):
        print("\n" + "="*30)
        print("🎧 분리 결과 확인")
        print("="*30)

        # 베이스 확인
        bass_path = os.path.join(output_dir, 'bass.wav')
        if os.path.exists(bass_path):
            print("\n🎸 Bass Track:")
            display(Audio(bass_path))

        # 기타 확인
        guitar_path = os.path.join(output_dir, 'guitar.wav')
        if os.path.exists(guitar_path):
            print("\n🎸 Electric Guitar Track:")
            display(Audio(guitar_path))

        print(f"\n✅ 전체 파일 위치: {output_dir}")
        print("왼쪽 파일 탐색기(폴더 아이콘)를 열어 다운로드 가능함")
    else:
        print("\n❌ 결과 폴더 생성 실패. 파일명에 특수문자가 있다면 영문으로 변경해 시도할 것")

# 함수 실행
if __name__ == "__main__":
    run_separator()

**1차 실험**

사용 음원: 일렉기타 2 트랙, 베이스 1 트랙, 직접 녹음

성공: 분리된 기타 트랙에서는 베이스를 찾기 어려움
아쉬움:
1. 분리된 베이스 트랙에서는 기타가 많이 섞여들어간 듯함
2. 초반부 베이스 없이 기타만 존재하는 구간에서 기타를 베이스로 오인함
3. 후반부 베이스 트랙에서 볼륨이 울렁거리며 분리도가 떨어지는 모습을 보임.  